# 07. Model Training GSV

Download a pre-trained object detection model from the TensorFlow 2 Object Garden, and train it to run detections via transfer learning
 

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Select a pre-trained model from the TensorFlow 2 Object garden at:
#  https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# You must specify both the name of the model and the download URL from the table on that page
pretrained_model_name = 'centernet_hg104_512x512_coco17_tpu-8'
pretrained_model_url  = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz'
frozen_model_name     = 'centernet_V1'

#pretrained_model_name = 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8'
#pretrained_model_url  = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz'
#frozen_model_name     = 'ssd_V1_V1'

#pretrained_model_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
#pretrained_model_url  = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'
#frozen_model_name     = 'faster_rcnn_V1'

#pretrained_model_name = 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'
#pretrained_model_url  = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'
#frozen_model_name     = 'ssd_resnet101_V1'

#pretrained_model_name = 'efficientdet_d1_coco17_tpu-32'
#pretrained_model_url  = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz'
#frozen_model_name     = 'efficientdet_D1_V1'

# Prefix that will be included as a suffix in the label map file and tfrecord train and test files
# Please the label map and tfrecords have already been created via the "05" Jupyter Notebook
dataset_version = 'V1'

## Code

In [2]:
# General imports
import os
import sys

from pathlib import Path

import wget

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Get root install path, a level above the minor_thesis folder from GitHub
install_path_root = Path(module_path_root).parent.absolute()

In [3]:
# Derived paths
api_dir              = os.path.join(install_path_root, 'TensorFlow', 'models')
annotations_dir      = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'annotations')
pretrained_model_dir = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'pre-trained-models')
model_dir            = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'models', pretrained_model_name)
frozen_model_dir     = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'models', frozen_model_name)
pipeline_config_file = os.path.join(model_dir, 'pipeline.config')

label_map_path       = os.path.join(annotations_dir, 'label_map_{0:s}.pbtxt'.format(dataset_version))
train_record_path    = os.path.join(annotations_dir, 'train_{0:s}.record'.format(dataset_version))
test_record_path     = os.path.join(annotations_dir, 'test_{0:s}.record'.format(dataset_version))

# Using a customized version of the training script to correct an error in the official version
#training_script      = os.path.join(api_dir, 'research', 'object_detection', 'model_main_tf2.py')
training_script      = os.path.join(module_path_root, 'contrib', 'model_main_tf2_mod.py')
freeze_script        = os.path.join(api_dir, 'research', 'object_detection', 'exporter_main_v2.py')

# Create directories if they do not already exist
Path(pretrained_model_dir).mkdir(parents=True, exist_ok=True)
Path(model_dir).mkdir(parents=True, exist_ok=True)

In [4]:
# Static configuration, a copy of the labels file built in notebook 05
labels = [
    {'name':'BikeLaneMarker', 'id':1}
]

In [5]:
# Download the requested pre-trained model
if os.name =='posix':
    !wget pretrained_model_url
    !mv {pretrained_model_name + '.tar.gz'} {pretrained_model_dir}
    !cd {pretrained_model_dir} && tar -zxvf {pretrained_model_name + '.tar.gz'}
if os.name == 'nt':
    wget.download(pretrained_model_url)
    !move {pretrained_model_name + '.tar.gz'} {pretrained_model_dir}
    !cd {pretrained_model_dir} && tar -zxvf {pretrained_model_name + '.tar.gz'}

  0% [                                                                    ]    1761280 / 1426099846

  0% [                                                                    ]    3743744 / 1426099846

  0% [                                                                    ]    5742592 / 1426099846

  0% [                                                                    ]    8019968 / 1426099846

  0% [                                                                    ]   10362880 / 1426099846

...

 20% [.............                                                       ]  287719424 / 1426099846

 20% [.............                                                       ]  290119680 / 1426099846

 20% [.............                                                       ]  292339712 / 1426099846

100% [....................................................................] 1426099846 / 1426099846        1 file(s) moved.


x centernet_hg104_512x512_coco17_tpu-8/
x centernet_hg104_512x512_coco17_tpu-8/checkpoint/
x centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint
x centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
x centernet_hg104_512x512_coco17_tpu-8/pipeline.config
x centernet_hg104_512x512_coco17_tpu-8/saved_model/
x centernet_hg104_512x512_coco17_tpu-8/saved_model/saved_model.pb
x centernet_hg104_512x512_coco17_tpu-8/saved_model/assets/
x centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/
x centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/variables.index


In [6]:
# Copy the default model config
if os.name =='posix':
    !cp {os.path.join(pretrained_model_dir, pretrained_model_name, 'pipeline.config')} {model_dir}
if os.name == 'nt':
    !copy {os.path.join(pretrained_model_dir, pretrained_model_name, 'pipeline.config')} {model_dir}  

        1 file(s) copied.


In [7]:
# Load the default config
config = config_util.get_configs_from_pipeline_file(pipeline_config_file)

In [8]:
# Create an object to merge config customizations into the default config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(pipeline_config_file, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [9]:
# Adjust the configuration
if pipeline_config.model.center_net.num_classes > 0:
    pipeline_config.model.center_net.num_classes = len(labels)
elif pipeline_config.model.ssd.num_classes > 0:
    pipeline_config.model.ssd.num_classes = len(labels)
    
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(pretrained_model_dir, pretrained_model_name, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_path
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [train_record_path]
pipeline_config.eval_input_reader[0].label_map_path = label_map_path
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [test_record_path]

In [10]:
# Write the customized configuration file to the model
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(pipeline_config_file, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

In [11]:
# Print the scripts needed to train, evaluate, or freeze the model
command_train  = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=".format(training_script, model_dir, pipeline_config_file)
command_eval   = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(training_script, model_dir, pipeline_config_file, model_dir)
command_freeze = "python {} --input_type=image_tensor -pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(freeze_script, pipeline_config_file, model_dir, frozen_model_dir)

print('Training command:')
print(command_train + '2000')
print('---')

print('Evaluation command:')
print(command_eval)
print('---')

print('Freeze command:')
print(command_freeze)

Training command:
python E:\Release\minor_thesis\contrib\model_main_tf2_mod.py --model_dir=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8 --pipeline_config_path=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8\pipeline.config --num_train_steps=2000
---
Evaluation command:
python E:\Release\minor_thesis\contrib\model_main_tf2_mod.py --model_dir=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8 --pipeline_config_path=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8\pipeline.config --checkpoint_dir=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8
---
Freeze command:
python E:\Release\TensorFlow\models\research\object_detection\exporter_main_v2.py --input_type=image_tensor -pipeline_config_path=E:\Release\TensorFlow\workspace\models\centernet_hg104_512x512_coco17_tpu-8\pipeline.config --trained_checkpoint_dir=E:\Release\TensorFlow\workspace\models\centerne